# Titanic - Machine Learning from Disaster


- Vamos utilizar os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - É um dataset de **competição**
    - O resultado é avaliado através da **acurácia**:
        - _"Sua pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido como acurácia."_

### Importando novamente as bases e fazendo o tratamento dos dados
- Vamos apenas replicar o que fizemos no **[primeiro arquivo dessa análise](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)** (para visualizar a parte 1 completa, visite esse link)

Importando o pandas

In [1]:
import pandas as pd

Visualizando a base de treino

In [2]:
treino = pd.read_csv('train.csv')

Visualizando a base de teste

In [3]:
teste = pd.read_csv('test.csv')

- **Fazendo o mesmo tratamento inicial que fizemos na aula anterior**

Eliminando as colunas com elevada cardinalidade

In [4]:
treino = treino.drop(['Name', 'Ticket', 'Cabin'], axis=1)
teste = teste.drop(['Name', 'Ticket', 'Cabin'], axis=1)


Usando a média para substituir valores nulos na coluna de idade

In [5]:
treino.loc[treino.Age.isnull(), 'Age'] = treino.Age.mean()
teste.loc[teste.Age.isnull(), 'Age'] = teste.Age.mean()


Tratando a coluna Embarked da base de treino usando a moda 

In [6]:
treino.loc[treino.Embarked.isnull(), 'Embarked'] = treino.Embarked.mode()[0]

E também a coluna Fare da base de teste usando a média

In [7]:
teste.loc[teste.Fare.isnull(), 'Fare'] = teste.Fare.mean()

### Podemos agora entender as colunas de texto
- Vamos agora **adicionar um novo tratamento das colunas de texto**

Verificando as colunas de texto na base de treino

In [9]:
treino.columns[treino.dtypes == 'object']

Index(['Sex', 'Embarked'], dtype='object')

Verificando os valores na coluna Sex

In [4]:
treino.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

e na coluna Embarked

In [5]:
treino.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

- Para tratar a coluna Sex, podemos criar uma nova coluna chamada **"Male_Check"** que **vai receber 1 se o gênero for masculino e 0 se o gênero for feminino**

Usando uma lambda function para fazer esse tratamento

In [11]:
treino['MaleCheck'] = treino.Sex.apply(lambda x: 1 if x == 'male' else 0)

Verificando os valores

In [12]:
treino[['Sex', 'MaleCheck']].value_counts()

Sex     MaleCheck
male    1            577
female  0            314
dtype: int64

- O próximo passo é fazer o mesmo para a base de teste

Usando uma lambda function para fazer esse tratamento

In [13]:
teste['MaleCheck'] = teste.Sex.apply(lambda x: 1 if x == 'male' else 0)

 Verificando os valores

In [14]:
teste[['Sex', 'MaleCheck']].value_counts()

Sex     MaleCheck
male    1            266
female  0            152
dtype: int64

- Agora, para tratar a coluna **Embarked** usando o **[OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)** que irá **criar uma nova coluna para cada um dos rótulos da coluna original**

Importando o OneHotEncoder

In [15]:
from sklearn.preprocessing import OneHotEncoder

Criando o encoder

In [16]:
ohe = OneHotEncoder(handle_unknown='ignore')

Fazendo o fit com os dados

In [17]:
ohe = ohe.fit(treino[['Embarked']])

Fazendo a transformação

In [18]:
ohe.transform(treino[['Embarked']]).toarray()

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

Transformando esse resultado em um DataFrame

In [21]:
ohe_df = pd.DataFrame(ohe.transform(treino[['Embarked']]).toarray(), columns=ohe.get_feature_names_out())
ohe_df.head()

,Embarked_C,Embarked_Q,Embarked_S
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0


Podemos agora adicionar essa coluna na nossa base de treino

In [22]:
treino = pd.concat([treino, ohe_df], axis=1)

Verificando os valores

In [23]:
treino[['Embarked', 'Embarked_C', 'Embarked_Q', 'Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0.0         0.0         1.0           646
C         1.0         0.0         0.0           168
Q         0.0         1.0         0.0            77
dtype: int64

- Agora vamos **fazer o mesmo para a base de teste usando o encoder ohe que criamos acima**

Transformando esse resultado em um DataFrame

Adicionando o resultado na base de teste

Verificando também os valores

### Usando essa nova base no modelo

Visualizando a base

Podemos então apagar essas duas colunas que já tratamos

- Podemos selecionar os mesmos modelos que vimos anteriormente (consulte arquivo da **[parte 1](https://github.com/lucaslealx/Titanic/blob/main/Parte1.ipynb)**)
    - **Árvore de classificação**
        - https://scikit-learn.org/stable/modules/tree.html#classification
    - **Classificação dos vizinhos mais próximos**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
- Antes de usar os algoritmos, precisamos separar a base de treino em **treino e validação**
    - Vamos fazer isso utilizando o **train_test_split**
        - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Importando o train_test_split

Separando a base de treino em X e y

Separando em treino e validação

- Para a **árvore de classificação**

Fazendo a importação

Criando o classificador

Fazendo o fit com os dados

Fazendo a previsão

- Para o **KNeighborsClassifier**

Importando

Criando o classificador

Fazendo o fit com os dados

Fazendo a previsão

- E para a **Regressão Logística**

Importando

Criando o classificador

Fazendo o fit com os dados

Fazendo a previsão

- Vamos novamente **avaliar os modelos**
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

- Avaliando a **acurácia**

Importando

Para a árvore

Para o knn

Para a regressão logística

- Avaliando a **matriz de confusão**

Importando

Para a árvore

Para o knn

Para a regressão logística

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

Visualizando o X_train

Visualizando a base de teste

Para a base de teste ser igual a base de treino, precisamos eliminar a coluna de id

Utilizando a regressão logística na base de teste

Criando uma nova coluna com a previsão na base de teste

Selecionando apenas a coluna de Id e Survived para fazer o envio

Exportando para um csv